In [9]:
import pandas as pd
import sqlalchemy

# Imports the method used for connecting to DBs
from sqlalchemy import create_engine

# Imports the methods needed to abstract classes into tables
from sqlalchemy.ext.declarative import declarative_base

# Allow us to declare column types
from sqlalchemy import Column, Integer, String, Float, Date, ForeignKey

#Allow us to establish session
from sqlalchemy.orm import Session

#Allows us to map DataFrame to sql database
from sqlalchemy.ext.automap import automap_base
from sqlalchemy import MetaData

# PyMySQL 
import pymysql
pymysql.install_as_MySQLdb()

In [10]:
measurements_df = pd.read_csv('clean_measurements.csv')
stations_df = pd.read_csv('clean_stations.csv')

In [11]:
measurements_df.head()

,station,date,prcp,tobs
0,USC00519397,2010-01-01,0.08,65
1,USC00519397,2010-01-02,0.00,63
2,USC00519397,2010-01-03,0.00,74
3,USC00519397,2010-01-04,0.00,76
4,USC00519397,2010-01-07,0.06,70


In [12]:
stations_df.head()

,station,name,latitude,longitude,elevation
0,USC00519397,"WAIKIKI 717.2, HI US",21.2716,-157.8168,3.0
1,USC00513117,"KANEOHE 838.1, HI US",21.4234,-157.8015,14.6
2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.5213,-157.8374,7.0
3,USC00517948,"PEARL CITY, HI US",21.3934,-157.9751,11.9
4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.4992,-158.0111,306.6


In [13]:
engine = create_engine("sqlite:///hawaii_3.sqlite")
conn = engine.connect()
Base = declarative_base()

In [14]:
class Measurements(Base):
    __tablename__ = 'measurements'
    id = Column(Integer, autoincrement = True, primary_key = True)
    station = Column(String)
    date = Column(Date)
    prcp = Column(Float)
    tobs = Column(Integer)
    
class Stations(Base):
    __tablename__ = 'stations'
    id = Column(Integer, autoincrement = True, primary_key = True)
    station = Column(String)
    name = Column(String)
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)

In [15]:
Base.metadata.create_all(engine)

In [16]:
engine.table_names()

['measurements', 'stations']

In [17]:
Base = automap_base()

Base.prepare(engine, reflect=True)

Base.classes.keys()

['measurements', 'stations']

In [18]:
stations_df.to_sql("stations", engine, if_exists='append', index=False)
measurements_df.to_sql("measurements", engine, if_exists='append', index=False)

In [19]:
engine.execute('SELECT * FROM measurements LIMIT 5').fetchall()

[(1, 'USC00519397', '2010-01-01', 0.08, 65),
 (2, 'USC00519397', '2010-01-02', 0.0, 63),
 (3, 'USC00519397', '2010-01-03', 0.0, 74),
 (4, 'USC00519397', '2010-01-04', 0.0, 76),
 (5, 'USC00519397', '2010-01-07', 0.06, 70)]

In [20]:
engine.execute('SELECT * FROM stations LIMIT 5').fetchall()

[(1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 (2, 'USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6),
 (3, 'USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0),
 (4, 'USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9),
 (5, 'USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6)]